In [1]:
!pip install -q google-generativeai beautifulsoup4 requests gradio

In [6]:
import requests
from bs4 import BeautifulSoup
import ssl
import socket
import json
import gradio as gr
from google.colab import userdata
from google import genai

API_KEY = userdata.get('GEMINI_API_KEY')
client = genai.Client(api_key=API_KEY)

def seo_checks(url):
    results = {
        "technical_seo": {},
        "on_page_seo": {},
        "security": {},
        "mobile_usability": {},
        "performance": {},
        "user_experience": {},
        "analytics_tracking": {}
    }

    try:
        r = requests.get(url, timeout=10)
        soup = BeautifulSoup(r.text, "html.parser")
        results["technical_seo"]["status_code"] = r.status_code

        results["on_page_seo"]["title"] = soup.title.string if soup.title else "Missing"
        meta_desc = soup.find("meta", attrs={"name": "description"})
        results["on_page_seo"]["meta_description"] = meta_desc["content"] if meta_desc and "content" in meta_desc.attrs else "Missing"

        results["security"]["https"] = url.startswith("https")

        images = soup.find_all("img")
        missing_alt = sum(1 for img in images if not img.get("alt"))
        results["on_page_seo"]["missing_image_alts"] = missing_alt

        internal_links = [a.get("href") for a in soup.find_all("a", href=True) if url in a.get("href")]
        results["on_page_seo"]["internal_links"] = len(internal_links)

        results["analytics_tracking"]["google_analytics"] = "UA-" in r.text or "gtag(" in r.text

    except Exception as e:
        results["error"] = str(e)

    return results

def audit_with_llm(url):
    if not url.strip():
        yield "⚠️ Please enter a valid URL"
        return

    yield "⏳ Processing..."
    python_results = seo_checks(url)

    prompt = f"""
    You are an SEO expert. Based on the JSON data below, evaluate the website in these categories and return a well-structured markdown report with headings and bullet points.

    ### Categories to Evaluate:
    1. Technical SEO & Crawlability
    2. On-Page SEO
    3. Security
    4. Mobile Usability and Responsiveness
    5. Performance & Core Web Vitals
    6. User Experience (UX/UI)
    7. Analytics & Tracking
    For each category:
        - Give a **score out of 10**
        - List **observations**
    For final website score:
        - Give a **score out of 70**
        - Give **recommendations**
    JSON Data:
    {json.dumps(python_results, indent=2)}
    """

    try:
        resp = client.models.generate_content(model="gemini-2.5-flash", contents=prompt)
        gemini_output = getattr(resp, "text", str(resp))
    except Exception as e:
        gemini_output = f"Gemini error: {e}"

    yield gemini_output

with gr.Blocks() as demo:
    with gr.Column(elem_classes="centered"):
        gr.Markdown("# Simple Website Auditor", elem_classes="centered")
        url_input = gr.Textbox(label="Website URL", placeholder="https://example.com", elem_classes="centered")
        with gr.Row(elem_classes="centered"):
            submit_btn = gr.Button("Submit", elem_classes="centered")
            clear_btn = gr.Button("Clear", elem_classes="centered")
        output_md = gr.Markdown()

        submit_btn.click(fn=audit_with_llm, inputs=url_input, outputs=output_md)
        clear_btn.click(lambda: ("", ""), None, [url_input, output_md])

demo.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://84570ee6b04ea5d187.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
